___

<a href='https://www.learntocodeonline.com/'> <img src='files/IMGs/learn to code online.png' /></a>
___

# What Are Django Models

In django, we use **models** to describe the data we need for our application. Django then uses these models to automatically setup and configure our database to store our data effectively.

Each model maps to a specific table in our database.

Django handles the interactions between our models and database on our behalf. Meaning? No need to write SQL statements directly.

# Creating Our User Database Model

The *user profile model* will be used to manage users in our system
- creating a registered user
- checking a user's credentials when they log in
- deleting users

Standard practice is to put all models in a `models.py` file in your app folder - in this example, it will be the `profiles-rest-api > src > profiles_project > profiles_api` folder.

<img src='files/IMGs/models/models01.png'>

As you noticed above, every app will have a *models.py* file created for you.

## Creating "User Profile" Model

Django out of the box has it's own user profile model we can use:  **django-admin**

But here, we will create a custom user model that will override django's built-in user model. This will provide us control over the fields required for the user profiles in our system.

Standard fields in django's original user model will be overwritten.

### Importing New Packages

**AbstractBaseUser** is the user base of the django standard user model. We will import and build on top of it.

`from django.contrib.auth.models import AbstractBaseUser`

This will be the base of our user model. We will also need to import the permissions mix-in:

`from django.contrib.auth.models import PermissionsMixin`

This allows us to add permissions to our user model.

In [ ]:
from django.contrib.auth.models import AbstractBaseUser
from django.contrib.auth.models import PermissionsMixin

### Creating The Model

Each model is defined by a class. It should inherit from both imported packages.

In [ ]:
class UserProfile(AbstractBaseUser, PermissionsMixin):
    """Represents a user profile in our system."""

Once the class is defined with docstring, we can start adding fields to the model. These **fields** are the data we want to collect and store within the database.

Be sure to check out the official django documentation for [model field reference](https://docs.djangoproject.com/en/1.11/ref/models/fields/). It has a list of all the different fields you can use in your system.

In [ ]:
    email = models.EmailField(max_length=255, unique=True)

The **max_length** attribute is **_required_** for all char fields. The max possible is 255. This retricts the length of the item, but also helps with a different validation on the field as well.

The **unique** attribute ensures there are no duplicate emails.

Now we'll add the name field.

In [ ]:
    name = models.CharField(max_length=255)

The next field will be to determine if a user is active in the system ... Meaning you can use this to disable user accounts.

This is a requirement when you add a custom user model to django.

This will use a BooleanField object type, and you should set the default to **True** if no other value is provided.

In [ ]:
    is_active = models.BooleanField(default=True)

The next field will be used to determine if a user is a Staff member. You will want this default to be **False**.

This is also required by django, so if you are doing a custom user model you will need to set up this field.

In [ ]:
    is_staff = models.BooleanField(default=False)

Once all fields are set for a model, you will need to create an **ObjectManager** - a class to help manage the user profile.

This is also required when substituting the user model for a custom one.

In [ ]:
    objects = UserProfileManager()

The **UserProfileManager** class will be created later.

There are a few more attributes to be set with the user profile model.

In [ ]:
    USERNAME_FIELD = 'email'

The standard django user model has a username field that users will login as - they're handle. We will be replacing this with their email address. (Generally easier for people to remember.)

Now to define the list of **_required_** fields that all users will use. It will be a list of strings and can be whatever fields you would like.

In [ ]:
    REQUIRED_FIELDS = ['name']

Email is not in this list because it is already required by the system automatically, since it is the username field.

Now it's time to create some helper functions for our model.

The first one will be a function we can use to get the full name of the user in the system. Since it is a **_class_** function it requires the _self_ as a required input parameter.

This class is used by the django-admin.

In [ ]:
    def get_name(self):
        """Used to get a user's full name."""
        
        return self.name

The next required class will return the "short name" of the user.

If you had a different type of name, used first and last name as separate fields, etc you could script something to accommodate for it. (Or even remove the @domain of the email.) For now, it will just return the name as is.

In [ ]:
    def get_short_name(self):
        """Used to get a user's short name."""
        
        return self.name

The next required function is the **__str__** method (with double underscores since it's a [dunder method](https://www.geeksforgeeks.org/dunder-magic-methods-python)). It is required in django so it knows how to return our object as a string.

In this instance, we will return the email since it will be unique.

In [ ]:
    def __str__(self):
        """Django uses this when it needs to convert an object to a string."""
        
        return self.email

## Add A User Model Manager

Since we created a unique user profile, we need to teach django how to use it.

In the same file, you will need to import the **BaseUSerManager** package.

In [ ]:
from django.contrib.auth.models import BaseUserManager

You will now begin building your **UserProfileManager** class.
- It must inherit from the **BaseUserManager** package you just added.
- It can be added above or below the **UserProfile** class. (I suggest above.)

_**NOTE:**  Be sure the class name of your user manager is the same as what you called it when created the user profile model._

There are many ways with which you can [customize your authentication](https://docs.djangoproject.com/en/1.11/topics/auth/customizing) in Django. If you would like more information on the normalize feature, **[click here](https://docs.djangoproject.com/en/1.11/topics/auth/customizing/#django.contrib.auth.models.BaseUserManager.normalize_email)**.

In [ ]:
class UserProfileManager(BaseUserMaanger):
    """Helps Django work with our custom user model."""
    
    # create user with model
    def create_user(self, email, name, password=None):
        """Creates a new user profile object."""
        
        # logic to create a new user in the system
        
        if not email:
            raise ValueError('Users must have an email address.')
        
        # normalize the email address - convert to lower case
        email = self.normalize_email(email)
        
        # create a new user profile object
        user = self.model(email=email, name=name)
        
        # set password - this encrypts the PW for us
        # password is converted to a hash stored in the DB - no clear text
        user.set_password(password)
        
        # save using same DB used when we created the UserProfileManager
        user.save(using=self._db)
        
        return user

The next class function will be used to tell Django how to create a Super User!

This user will have full control of everything.

In [ ]:
    # create super user
    def create_superuser(self, email, name, password):
        """Creates and saves a new superuser with given details."""
        
        user = self.create_user(email, name, password)
        
        user.is_superuser = True
        user.is_staff = True
        
        user.save(using=self._db)
        
        return user

# Set Our Custom Django User Model

We ahve to configure Django to know how to use these models and user manager.

This is done in the settings file. This is located in the project folder.

In this instance it will be the following location:
`profiles-rest-api > src > profiles_project > profiles_project > settings.py`

<img src='files/IMGs/models/models02.png'>

At the bottom of the file must add a new attribute to tell Django to use our custom user model instead of it's built-in default user model.

In [ ]:
AUTH_USER_MODEL = "profiles_api.UserProfile"

The string that is assigned to the attribute is the model you wish for it to use. In order to do so, it must be the name of the app, dot notation, then the class as shown above.

# Create Migrations & Sync Database

Django has an awesome feature called **database migrations** - which can be used to automatically create and make changes to our DB based on what we change in the models.

It works by looking at the current stated of the DB, the current state of our models, and seeing what the differences are.

You create the migrations using the management command that comes with Django.

## Create Migrations

In the vagrant server ...

1. In git bash go to project directory:  `cd workspace/PROJECTNAME`
2. `vagrant up`
3. `vagrant ssh`
4. `workon VIRTUALENVNAME` (such as **profiles_api**)
5. `cd /vagrant/src/profiles_project`
6. `python manage.py makemigrations`

This command will check our models, the database, and make a migration file that contains all of the changes from what's existing in the database & what has been changed in the models.

If this is a first time run, it will just set the database up as is, since no changes to be done.

<img src='files/IMGs/db/db01.png'>

If there is an error, it will tell you and you will need to update your code.

When you look back in Atom, you will see that a new folder in the changed app directory has been created.

<img src='files/IMGs/db/db02.png'>

You will also see there is a new file called *0001_initial.py* - which is automatically created by Django. It tells Django how to set up our database



## Run Migrations

Initially this will be to create the database. Subsequent runs will update as needed.

In the same running vagrant server, run:  `python manage.py migrate`

This will go through all of our DB migrations and run them on the DB.

<img src='files/IMGs/db/db03.png'>

As you can see, a bunch of things were created. The other files around our migration final (the initial one) are migration files that are included with the different Django apps that we're using.

- using the Django auth app creates the auth table
- using the Django sessions creates a sessions table for the admin

This has also created a SQLite file.

**DO NOT** try to open this, as it is generally a large binary file and could crash the editor.

<img src='files/IMGs/db/db04.png'>

If you continue to run the migrate command, it will tell you that there were no migrations to apply.

<img src='files/IMGs/db/db05.png'>

The reason is because everything has been applied to the database - no need to apply more.



## Commit To Git

In your **git bash** program ...

1. go to project directory:  `cd workspace/PROJECTNAME` (in this example **profiles-rest-api**)
2. Call `git add .`
3. Call `git commit -am "Added a custom user profile model, a manager, and the migrations for our DB."